In [8]:
library("dplyr")

## Inputs: 

1. Set of nodes *Z*
2. Ordering of the nodes
3. Upper bound on the number of parents (for each node) *u*
4. Database *D* of m cases

In [17]:
x1 <- c(1,1,0,1,0,0,1,0,1,0)
x2 <- c(0,1,0,1,0,1,1,0,1,0)
x3<- c(0,1,1,1,0,1,1,0,1,0)

D <- data.frame(x1, x2, x3)

print(D)


   x1 x2 x3
1   1  0  0
2   1  1  1
3   0  0  1
4   1  1  1
5   0  0  0
6   0  1  1
7   1  1  1
8   0  0  0
9   1  1  1
10  0  0  0


In [71]:
N_tensor <- function(D, i, V.i, pi.i){
    result <- D%>% group_by_at(names(D)[pi.i]) %>% mutate(j_counts=n())
    
        for (k in V.i){
            result.k <- D %>% filter(D[,i]==k) %>% group_by_at(names(D)[pi.i]) %>% mutate(k=n())
            colnames(result.k)[length(colnames(result.k))]<- k
            result<-left_join(result, result.k, by=names(D))
        }
        result[is.na(result)] <- 0
    result<-result %>% group_by_at(names(result)[pi.i]) %>% filter(row_number() == 1)
    print(result)
    return(result)
}

g <- function(V.i, r.i, N.tensor){
    N.ij <- N.tensor$j_counts
    print(N.ij)
    q.i <- length(N.ij) # number of unique combinations of the parents realizations
    result <- 1
    for (j in 1:q.i) {
        a <- 1
        for (k in V.i){a <- a*factorial(N.tensor[j,as.character(k)])}
        result <- result*a*factorial(r.i-1)/factorial(N.ij[j]+r.i-1)
    }
    return(result)
}


k2 <- function(Z, u, D){
    V <- unname(sapply(D, unique)) #it's a matrix
    r <- unname(sapply(D, n_distinct)) #it's a vector
    parents <- list()
    for (i in 1:length(Z)){
        cat('\n iteration', i)
        pi.i <- NULL 
        Nijk <- N_tensor(D, Z[i], V[,i], pi.i)
        P_old <- g(V[,i],r[i], Nijk)
        proceed <- TRUE
        while (proceed & (length(pi.i)<u & i>1)){
            pred.i <- setdiff(Z[1:i-1], pi.i)
            P <- NULL
            for (node in pred.i){
                new_set <- union(pi.i, node)
                Nijk <- N_tensor(D, Z[i], V[,i], new_set)
                P<- c(P, g(V[,i], r[i], Nijk))
            }
            P<-unlist(P)
            print(paste("Pold:", P_old))
            print(paste("P:", P))
            if (max(P) > P_old){
                max.indx <- which.max(P)
                P_old <- P[max.indx]
                pi.i <- union(pi.i, pred.i[max.indx])
            }
            else {proceed = FALSE}
        }
        append(parents, list(pi.i))
    } 
    return(parents)
}



In [72]:
k2(c(1,2,3), u=3, D)


 iteration 1# A tibble: 1 × 6
     x1    x2    x3 j_counts   `1`   `0`
  <dbl> <dbl> <dbl>    <int> <int> <int>
1     1     0     0       10     5     0
[1] 10

 iteration 2# A tibble: 1 × 6
     x1    x2    x3 j_counts   `0`   `1`
  <dbl> <dbl> <dbl>    <int> <int> <int>
1     1     0     0       10     5     0
[1] 10
# A tibble: 2 × 6
# Groups:   x1 [2]
     x1    x2    x3 j_counts   `0`   `1`
  <dbl> <dbl> <dbl>    <int> <int> <int>
1     1     0     0        5     1     0
2     0     0     1        5     4     0
[1] 5 5
[1] "Pold: 3.00625300625301e-06"
[1] "P: 4.62962962962963e-05"
[1] "Pold: 4.62962962962963e-05"
[1] "P: "


Warning message in max(P):
“nessun argomento non-mancante al massimo; si restituisce -Inf”



 iteration 3# A tibble: 1 × 6
     x1    x2    x3 j_counts   `0`   `1`
  <dbl> <dbl> <dbl>    <int> <int> <int>
1     1     0     0       10     4     0
[1] 10
# A tibble: 2 × 6
# Groups:   x1 [2]
     x1    x2    x3 j_counts   `0`   `1`
  <dbl> <dbl> <dbl>    <int> <int> <int>
1     1     0     0        5     1     0
2     0     0     1        5     0     2
[1] 5 5
# A tibble: 2 × 6
# Groups:   x2 [2]
     x1    x2    x3 j_counts   `0`   `1`
  <dbl> <dbl> <dbl>    <int> <int> <int>
1     1     0     0        5     4     0
2     1     1     1        5     0     5
[1] 5 5
[1] "Pold: 6.01250601250601e-07"
[1] "P: 3.85802469135802e-06" "P: 0.00555555555555556" 
# A tibble: 4 × 6
# Groups:   x2, x1 [4]
     x1    x2    x3 j_counts   `0`   `1`
  <dbl> <dbl> <dbl>    <int> <int> <int>
1     1     0     0        1     1     0
2     1     1     1        4     0     4
3     0     0     1        4     0     1
4     0     1     1        1     0     1
[1] 1 4 4 1
[1] "Pold: 0.00555555555555556"
[

list()

In [70]:
?merge

In [9]:
?dplyr::mutate

In [50]:
?dplyr::full_join

In [62]:
pi.i<-c(2,3)
names(D)[pi.i]

[1] "x2" "x3"